# Connecting to a JBoss Virtual Database Using JDBC Driver

**NOTE:** In order for JayDeBeApi library to work with the Red Hat JBoss JDBC driver, have to install JPype1 version 0.6.3 instead of the latest version 0.7.  Otherwise, you'll get ```convertStrings``` error per this GitHub [issue](https://github.com/jpype-project/jpype/issues/498).

Therefore, you can install JayDeBeApi and jpype1 version 0.6.3 with conda command by doing:<br>
    ```conda install JayDeBeApi jpype1==0.6.3```

You also need to set the ```JAVA_HOME``` environment variable and know the path to the .jar file containing the teiid driver.

In [ ]:
!echo %JAVA_HOME%

In [ ]:
import jaydebeapi
import pandas as pd
from getpass import getpass

In [ ]:
username = getpass("Enter your user name: ")
password = getpass("Enter your password: ")

In [ ]:
sql = """
SELECT
    *
FROM
    some_table
"""

df = pd.read_sql(sql, cnxn, index_col=None)

cnxn.close()

### Usually best practice to use a context manager when connecting to a database to ensure database connection will be closed

jaydebeapi library does not support context management, so we'll use contextmanager decorator from the contextlib module to enable it

In [ ]:
from contextlib import contextmanager

In [ ]:
@contextmanager
def db_connect(db, user, pwd):
    conn = jaydebeapi.connect('com.ibm.db2.jcc.DB2Driver', f'jdbc:teiid:{db}@mm://some_host:31000',
                    [user, pwd], jars=['D:/JDBC_Drivers/mainframe_db2/db2jcc4.jar'])
    try:
        yield conn
    finally:
        conn.close()

In [ ]:
with db_connect('your_database', username, password) as conn:
    df = pd.read_sql(sql, conn, index_col=None)

In [ ]:
df.head()